In [36]:
import gym

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
from collections import namedtuple

In [37]:
ENV_NAME = 'Pendulum-v0'
env = gym.make(ENV_NAME)
GAMMA = .9
LR_actor = .01
LR_critic = .01
EXPERIENCE_CAPACITY = 2000
TARGET_UPDATE_FREQUENCY = 100
MAX_EPI = 400
MAX_STEP = 200
BATCH_SIZE = 32
EPSILON = .1
N_STATES = env.observation_space.shape[0]
N_ACTIONS = env.action_space.shape[0]

[2017-08-05 15:20:25,913] Making new env: Pendulum-v0


In [35]:
# initialization
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
Tensor = FloatTensor

Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state'))

NameError: name 'namedtuple' is not defined

In [56]:
# classes
class Experience(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.mem = []
        self.pos = 0
    
    def push(self, o, a, r, o_next):
        if len(self.mem) < self.capacity:
            self.mem.append(None)
        self.mem[self.pos] = Transition(o, a, r, o_next)
        self.pos = (self.pos + 1) % self.capacity
    
    def sample(self, size):
        return random.sample(self.mem, min(size, len(self.mem)))
    
class Critic(nn.Module):
    def __init__(self):
        super(Critic, self).__init__()
        self.fcS = nn.Linear(N_STATES, 5)
        self.fcS.weight.data.normal_(0, 0.1)
        self.fcA = nn.Linear(N_ACTIONS, 5)
        self.fcA.weight.data.normal_(0, 0.1)
        self.fcO = nn.Linear(10, 1)
        self.fcO.weight.data.normal_(0, 0.1)
        
    def forward(self, x):
        print x
        print x
        s = self.fcS(x[:, :N_STATES])
        s = F.relu(s)
        a = self.fcA(x[:, -N_ACTIONS:])
        a = F.relu(a)
        o = self.fcO(torch.cat([s,a], 1))
        return o

In [7]:
target_actor = Actor()
target_critic = Critic()
actor = Actor()
critic = Critic()
if use_cuda:
    target_actor.cuda()
    target_critic.cuda()
    actor.cuda()
    critic.cuda()
    

for epi in xrange(MAX_EPI):
    o = env.reset()
    for t in xrange(MAX_STEP):
        a = choose_action(actor, seed)
        o_, r, done, info = env.step(a)
        exp.push(o, a, r, o_)
        
        update_actor_critic(target_actor, target_critic, \
                           actor, critic, exp)
        
        if update_counter % TARGET_UPDATE_FREQUENCY == 0:
            update_target(target_actor, target_critic, \
                         actor, critic)
        
        o = o_
        if done:
            break

[2017-08-05 14:40:17,470] Making new env: Pendulum-v0


3


In [58]:
critic = Critic().cuda()
s = Variable(Tensor([1,2,3,4]).unsqueeze(0))
#s.unsqueeze(0)
print critic(s)

Variable containing:
 1  2  3  4
[torch.cuda.FloatTensor of size 1x4 (GPU 0)]

Variable containing:
 1  2  3  4
[torch.cuda.FloatTensor of size 1x4 (GPU 0)]

Variable containing:
-0.2195
[torch.cuda.FloatTensor of size 1x1 (GPU 0)]

